# Residual Network (ResNet)
---

## 目的
ResNetの構造 (スキップ構造) の仕組みを理解する．
ResNetを用いてCIFAR10データセットに対する物体認識を行う．


## Residual Network (ResNet)
ResNetの説明．


## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．
`pickle`はPythonのリストや辞書などのオブジェクトを保存・読み込みを行うためのライブラリです．今回はCIFAR-10データセットを読み込むために使用します・

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

## データセットの読み込み
実験に使用するCIFAR-10データセットを読み込みます．

まず，CIFAR-10データセットをダウンロードします．

In [ ]:
# CIFAR-10データセットのダウンロード
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                    download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                      shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                   download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                     shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

次に，ダウンロードしたデータセットを読み込みます．

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

## ネットワークモデルの定義
次に，CNNを定義します．

まずはじめに，ネットワークの定義に必要な関数を定義します．

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

`im2col`およびその逆の変換の`col2im`も定義を行います．

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

畳み込みおよびプーリングの処理は煩雑になってしまうため，関数として定義します．

In [ ]:
epochs = 2
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

次に，上で定義した関数を用いてネットワークを定義します．
ここでは，畳み込み層，中間層，出力層から構成されるCNNとします．

入力画像のチャンネル数と，畳み込みのカーネルサイズ，畳み込みのカーネル数を引数として指定します．
さらに，中間層，出力層のユニット数は引数として与え，それぞれ`hidden_size`, `output_size`とします．
そして，`__init__`関数を用いて，ネットワークのパラメータを初期化します．
`w1`, `w2`, `w3`は各層の重みで，`b1`, `b2`, `b3`はバイアスを表しています．
重みは`randn`関数で，標準正規分布に従った乱数で生成した値を保有する配列を生成します．
バイアスは`zeros`関数を用いて，要素が全て0の配列を生成します．

そして，`forward`関数で，データを入力して結果を出力するための演算を定義します．

次に，`backward`関数ではパラメータの更新量を計算します．
まず，ネットワークの出力結果と教師ラベルから，誤差`dy`を算出します．
この時，教師ラベルをone-hotベクトルへ変換し，各ユニットの出力との差を取ることで，`dy`を計算しています．
その後，連鎖律に基づいて，出力層から順番に勾配を計算していきます．
このとき，パラメータの更新量を`self.grads`へ保存しておきます．

最後に`update_parameters`関数で，更新量をもとにパラメータの更新を行います．

In [ ]:
class CNN:
    
    def __init__(self, n_channels=3, filter_size=3, num_kernel=64, hidden_size=128, output_size=10, w_std=0.01):
        
        # convolutional layer
        self.w1 = w_std * np.random.randn(num_kernel, n_channels, filter_size, filter_size)
        self.b1 = np.zeros(num_kernel)
        # hidden layer
        pooled_feature_size = int(num_kernel * (32 / 2) * (32 / 2))
        self.w2 = w_std * np.random.randn(pooled_feature_size, hidden_size)
        self.b2 = np.zeros(hidden_size)
        # output layer
        self.w3 = w_std * np.random.randn(hidden_size, output_size)
        self.b3 = np.zeros(output_size)
        # dict. for gradients
        self.grads = {}
        
    def forward(self, x):
        self.h1, self.h1_col, self.h1_col_w = conv(x, self.w1, self.b1, stride=1, padding=1)
        self.h2 = relu(self.h1)
        self.h3, self.h3_argmax = maxpool(self.h2, pool_size=2, stride=2, padding=0)
        self.h4 = np.dot(self.h3.reshape(self.h2.shape[0], -1), self.w2) + self.b2
        self.h5 = relu(self.h4)
        self.h6 = np.dot(self.h5, self.w3) + self.b3
        return self.h6    
        
    def backward(self, x, t):
        batch_size = x.shape[0]
        
        # forward  #####
        _ = self.forward(x)
        y = softmax(self.h6)
        
        # backward #####
        self.grads = {}
        
        t = np.identity(10)[t]
        
        dy = (y - t) / batch_size
        
        # output layer
        d_h5 = np.dot(dy, self.w3.T)
        self.grads['w3'] = np.dot(self.h5.T, dy)
        self.grads['b3'] = np.sum(dy, axis=0)
        
        # relu
        d_h4 = relu_grad(self.h4) * d_h5
        
        # hidden layer
        d_h3 = np.dot(d_h4, self.w2.T)
        self.grads['w2'] = np.dot(self.h3.T, d_h4)
        self.grads['b2'] = np.sum(d_h4, axis=0)
        
        # maxpool
        d_h3 = d_h3.reshape(self.h3.shape)
        d_h2 = maxpool_grad(d_h3, self.h2, self.h3_argmax, p_size=2, stride=2, padding=0)
        
        # relu
        d_h1 = relu_grad(self.h1) * d_h2
        
        # convolution
        _, self.grads['w1'], self.grads['b1'] = conv_grad(d_h1, x, self.h1_col, self.h1_col_w, self.w1, self.b2, stride=1, padding=1)
        
    def update_parameters(self, lr=0.1): 
        self.w1 -= lr * self.grads['w1']
        self.b1 -= lr * self.grads['b1']
        self.w2 -= lr * self.grads['w2'].reshape(self.w2.shape)
        self.b2 -= lr * self.grads['b2']
        self.w3 -= lr * self.grads['w3']
        self.b3 -= lr * self.grads['b3']

## ネットワークの作成と学習の準備

読み込んだCIFAR10データセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を100，学習エポック数を10とします．

学習データは毎回ランダムに決定するため，numpyの`permutation`という関数を利用します．
各更新において，学習用データと教師データをそれぞれ`x_batch`と`y_batch`とします．
学習モデルに`x_batch`を与えて，`h`を取得します．
取得した`h`は精度および誤差を算出するための関数へと入力され，値を保存します．
そして，誤差を`backward`関数で逆伝播し，`update_parameters`でネットワークの更新を行います．

In [ ]:
model = CNN(n_channels=3, filter_size=3, num_kernel=64, hidden_size=256, output_size=10)

## 学習
学習したネットワークを用いて，テストデータに対する認識律の確認を行います．

In [ ]:
num_train_data = x_train.shape[0]
batch_size = 100
epoch_num = 10

iteration = 1
start = time()
for epoch in range(1, epoch_num + 1):
    sum_accuracy = 0.0
    sum_loss= 0.0
    
    perm = np.random.permutation(num_train_data)
    for i in range(0, num_train_data, batch_size):
        x_batch = x_train[perm[i:i+batch_size]]
        y_batch = y_train[perm[i:i+batch_size]]
        
        h = model.forward(x_batch)
        sum_accuracy += multiclass_classification_accuracy(h, y_batch)
        loss = softmax_cross_entropy(h, y_batch)
        sum_loss += loss
        
        model.backward(x_batch, y_batch)
        model.update_parameters(lr=0.1)
        
        if iteration % 100 == 0:
            print("iteration: {}, loss: {}".format(iteration, loss))
        
        iteration += 1

    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed time: {}".format(epoch,
                                                                                 sum_loss / num_train_data,
                                                                                 sum_accuracy / num_train_data,
                                                                                 time() - start))

## テスト
学習したネットワークを用いて，テストデータに対する認識率の確認を行います．

In [ ]:
count = 0
num_test_data = x_test.shape[0]

for i in range(num_test_data):
    x = np.array([x_test[i]], dtype=np.float32)
    t = y_test[i]
    y = model.forward(x)
    pred = np.argmax(y.flatten())
    
    if pred == t:
        count += 1

print("test accuracy: {}".format(count / num_test_data))